# original data

In [1]:
import pandas as pd
import numpy as np

step to create the data:
- make sure that in the folder `raw_data` there is a file `NYCTAXI202401-202403_OD.zip`
- cd to the folder `raw_data`
- linux shell `unzip NYCTAXI202401-202403_OD.zip`
- run the following notebook
- the output will be saved in the folder `data`, which will have two folders `total` and `timeperiod`.

In [2]:
import os
if not os.path.exists("data"):
    os.makedirs("data")
if not os.path.exists("data/timeperiod"):
    os.makedirs("data/timeperiod")
if not os.path.exists("data/total"):
    os.makedirs("data/total")

In [3]:
geo = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.geo')
print(geo.shape)
geo.head( )

(263, 3)


,geo_id,type,coordinates
0,0,Polygon,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
1,1,MultiPolygon,"[[[['-73.81768667', '40.61493300'], ['-73.8165..."
2,2,Polygon,"[[['-73.84240629', '40.86847474'], ['-73.84280..."
3,3,Polygon,"[[['-73.97464721', '40.71884235'], ['-73.97347..."
4,4,Polygon,"[[['-74.17225508', '40.56170484'], ['-74.17348..."


In [4]:
geo["type"].value_counts()

Polygon         240
MultiPolygon     23
Name: type, dtype: int64

In [5]:
rel = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.rel')
print(rel.shape)
rel.head()

(69169, 5)


,rel_id,type,origin_id,destination_id,cost
0,0,geo,0,0,0.0
1,1,geo,0,1,31349.0
2,2,geo,0,2,33747.0
3,3,geo,0,3,17205.0
4,4,geo,0,4,14927.0


In [6]:
od = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.od')#.drop(["type"],axis=1)
print(od.shape)
od.head()

(150995927, 6)


,dyna_id,type,time,origin_id,destination_id,flow
0,0,state,2024-01-01T00:00:00Z,0,0,0.0
1,1,state,2024-01-01T01:00:00Z,0,0,0.0
2,2,state,2024-01-01T02:00:00Z,0,0,0.0
3,3,state,2024-01-01T03:00:00Z,0,0,0.0
4,4,state,2024-01-01T04:00:00Z,0,0,0.0


In [7]:
# od["flow"].value_counts().to_frame().reset_index()

# Get two kind of network

## Network group by time interval

        早高峰时段（7:00 - 9:00）	上班、上学的高峰期。
        上午时段（9:00 - 12:00）	早高峰过后到中午的时间段。
        午间时段（12:00 - 14:00）	中午吃饭和午休时间。
        下午时段（14:00 - 17:00）	午休结束到晚高峰前的时间段。
        晚高峰时段（17:00 - 19:00）	下班、下学的高峰期。
        晚间时段（19:00 - 7:00）	晚高峰过后到第二天早高峰前的时间段。


In [8]:
od["time"] = pd.to_datetime(od["time"])
od["hour"] = od["time"].dt.hour
od

69169的od对

In [ ]:
# Define the boundaries of the time periods
bins = [0, 7, 9, 12, 14, 17, 19, 24]

# Define the labels for the time periods
labels = ['Night', 'Morning Rush', 'Morning', 'Noon', 'Afternoon', 'Evening Rush', 'Night']
# num_labels = [0,1,2,3,4,5,0]

tmp = od.copy(deep =True)
tmp['time_period'] = pd.cut(tmp['hour'], bins=bins, labels=labels, right=False,ordered=False)
# tmp["time_period"] = tmp["time_period"].astype(str)

In [ ]:
# tmp_morningrush = tmp.groupby("time_period").get_group("Morning")
# # tmp.info()
# tmp_morningrush.groupby(["origin_id","destination_id"]).size()

In [ ]:
for i in labels:
    tmp_group = tmp.groupby("time_period").get_group(i)
    tmp_group.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()
    tmp_group["cost"] = rel["cost"]
    tmp_group.to_csv(f"data/timeperiod/NYCTAXI_OD_{i}.csv")

In [ ]:
# od_flow_nozero = od[od["flow"]>0].reset_index(drop=True)

# # Define the boundaries of the time periods
# bins = [0, 7, 9, 12, 14, 17, 19, 24]

# # Define the labels for the time periods
# labels = ['Night', 'Morning Rush', 'Morning', 'Noon', 'Afternoon', 'Evening Rush', 'Night']
# # num_labels = [0,1,2,3,4,5,0]

# # Create a new column that groups hours into time periods
# od_flow_nozero['time_period'] = pd.cut(od_flow_nozero['hour'], bins=bins, labels=labels, right=False,ordered=False)

# # Use the groupby function to group by the new column
# od_flow_nozero_agg = od_flow_nozero.groupby(["time_period","origin_id","destination_id"]).agg({"flow": ["count","sum","mean"]}).reset_index()

# od_flow_nozero_agg = od_flow_nozero_agg.loc[od_flow_nozero_agg["flow"]["count"]!=0,:]

# od_flow_nozero_f = od_flow_nozero_agg.copy(deep=True)
# od_flow_nozero_f.columns = ["time_period","origin_id","destination_id","count","sum","mean"]
# od_flow_nozero_f.drop(["count","sum"],axis=1,inplace=True)
# od_flow_nozero_f.rename(columns={"mean":"mean_flow"},inplace = True)
# od_flow_nozero_f

# od_flow_nozero_f.to_csv("data/NYCTAXI_OD_timeperiod.csv",index=False)


## Total flow network

agg od by time   -> data/NYCTAXI_OD_agg.csv

In [ ]:
od_agg = od.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()

In [ ]:
od_agg

In [ ]:
# od_agg.to_csv("data/NYCTAXI_OD_agg.csv",index=False)

join od_agg and rel

In [ ]:
# od_agg = pd.read_csv("data/NYCTAXI_OD_agg.csv")
od_f = od_agg.copy(deep = True)

od_f["cost"] = rel["cost"]

In [ ]:
od_f.to_csv("data/total/NYCTAXI_OD_Total.csv",index=False)

In [ ]:
od_f.head()

## add location to the od_f --- not used

In [ ]:
# od_f_merge = pd.merge(od_f,geo,left_on="origin_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
#     .rename(columns={"coordinates":"origin_coordinates"})
# od_f_merge

In [ ]:
# od_f_merge = pd.merge(od_f_merge,geo,left_on="destination_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
#     .rename(columns={"coordinates":"destination_coordinates"})
# od_f_merge